In [2]:
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras._impl.keras.datasets.cifar10 import load_data
tf.reset_default_graph()
sess = tf.InteractiveSession()

(x_train, y_train), (x_test,y_test) = load_data()

def conv2d(x, w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

def next_batch(num,data,labels):
    idx = np.arange(0,len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
labels = tf.placeholder(tf.float32, [None,10])

x_image = x
global_step = tf.Variable(0, trainable=False, name='global_step')

with tf.name_scope('conv1'):
    w_conv1 = tf.Variable(tf.truncated_normal([7,7,3,128],stddev=0.01),name='weight')
    b_conv1 = tf.Variable(tf.zeros([128]),name='biases')
    h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
    print(h_conv1.shape)

with tf.name_scope('conv2_three_3by3_b/n'):
    with tf.name_scope('bottleneck'):
        w_conv2_b = tf.Variable(tf.truncated_normal([1,1,128,64],stddev=0.01),name='weight')
        b_conv2_b = tf.Variable(tf.zeros([64]),name='biases')
        h_conv2_b = tf.nn.relu(conv2d(h_conv1,w_conv2_b)+b_conv2_b)
    with tf.name_scope('conv2_1'):
        w_conv2_1 = tf.Variable(tf.truncated_normal([3,3,64,64],stddev=0.01),name='weight')
        b_conv2_1 = tf.Variable(tf.zeros([64]),name='biases')
        h_conv2_1 = tf.nn.relu(conv2d(h_conv2_b,w_conv2_1)+b_conv2_1)
    with tf.name_scope('conv2_2'):
        w_conv2_2 = tf.Variable(tf.truncated_normal([3,3,64,64],stddev=0.01),name='weight')
        b_conv2_2 = tf.Variable(tf.zeros([64]),name='biases')
        h_conv2_2 = tf.nn.relu(conv2d(h_conv2_1,w_conv2_2)+b_conv2_2)
    with tf.name_scope('conv2_3'):
        w_conv2_3 = tf.Variable(tf.truncated_normal([3,3,64,64],stddev=0.01),name='weight')
        b_conv2_3 = tf.Variable(tf.zeros([64]),name='biases')
        h_conv2_3 = tf.nn.relu(conv2d(h_conv2_2,w_conv2_3)+b_conv2_3)
    with tf.name_scope('restore'):
        w_conv2_r = tf.Variable(tf.truncated_normal([1,1,64,128],stddev=0.01),name='weight')
        b_conv2_r = tf.Variable(tf.zeros([128]),name='biases')
        h_conv2_r = tf.nn.relu(conv2d(h_conv2_3,w_conv2_r)+b_conv2_r)
        print(h_conv2_r.shape)
        p_conv = max_pool_2x2(h_conv2_r)
        print(p_conv.shape)

with tf.name_scope('fc'):
    p_conv_flat = tf.reshape(p_conv, [-1,16*16*128])
    w_fc = tf.Variable(tf.truncated_normal([16*16*128,10],stddev=0.01),name='weight')
    b_fc = tf.Variable(tf.zeros([10]),name='biases')
    logits = tf.matmul(p_conv_flat,w_fc)+ b_fc
    print(logits.shape)

with tf.name_scope('softmax'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
    train_op = tf.train.AdamOptimizer(1e-4).minimize(loss,global_step=global_step)
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('loss',loss)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs/1by1',sess.graph)
sess.run(tf.global_variables_initializer())
for i in range(500):
    batch = next_batch(100,x_train,y_train_one_hot.eval())
    if(i%50 == 0 or i==499):
        test_batch = next_batch(500,x_test,y_test_one_hot.eval())
        train_accuracy = accuracy.eval(feed_dict={x:batch[0],labels:batch[1]})
        print("step %d, accuracy %g"%(i,train_accuracy))
        summary, test_accuracy = sess.run([merged, accuracy],feed_dict={x:test_batch[0],labels:test_batch[1]})
        print("step %d, test_accuracy %g"%(i,test_accuracy))
        writer.add_summary(summary,global_step=sess.run(global_step))
    start = time.time()
    sess.run(train_op,feed_dict={x:batch[0],labels:batch[1]})
    duration =time.time() - start
    if(i%50 == 0 or i==499):
        print("training time %.3f sec"%(duration))

(?, 32, 32, 128)
(?, 32, 32, 128)
(?, 16, 16, 128)
(?, 10)
step 0, accuracy 0.08
step 0, test_accuracy 0.096
training time 1.487 sec
step 50, accuracy 0.14
step 50, test_accuracy 0.192
training time 1.460 sec
step 100, accuracy 0.37
step 100, test_accuracy 0.262
training time 1.596 sec
step 150, accuracy 0.28
step 150, test_accuracy 0.306
training time 1.444 sec
step 200, accuracy 0.24
step 200, test_accuracy 0.356
training time 1.516 sec
step 250, accuracy 0.39
step 250, test_accuracy 0.342
training time 1.479 sec
step 300, accuracy 0.31
step 300, test_accuracy 0.372
training time 1.505 sec
step 350, accuracy 0.37
step 350, test_accuracy 0.402
training time 1.476 sec
step 400, accuracy 0.42
step 400, test_accuracy 0.4
training time 1.487 sec
step 450, accuracy 0.46
step 450, test_accuracy 0.416
training time 1.478 sec
step 499, accuracy 0.51
step 499, test_accuracy 0.448
training time 1.690 sec
